# Numpy

In [ ]:
import numpy as np

The core of the `numpy` package is the `array` class. Let's examine that first. We can make an array out of a sequence, like a list.

In [ ]:
d = [1, 2, 3, 4, 5]
np.array(d)

### data types

Unlike lists, arrays must be homogeneous, in that the data types of each element must be the same. The data type of the array is upcast to be able to represent all of the data. So, if only one element is a float, all elements will be converted to floats.

In [ ]:
d = [1, 2, 3.1415, 4, 5]
np.array(d)

You can query the datatype by examining the `dtype` attribute of the array.

In [ ]:
d = [1, 2, 3.1415, 4, 5]
arr = np.array(d)
arr.dtype

Array types may be defined explicity in the call

In [ ]:
arr = np.array([1, 2, 3, 4, 5], dtype='float32')
arr

Complex numbers are noted with a lowercase `j` or uppercase `J`, like this

In [ ]:
cmplx = np.array([1.0+2.0j, 3.0+4.0J])
print(cmplx)
cmplx.dtype

As we have seen before, arrays are like multidimensional sequences. We can create a 2D array by supplying a list of lists as the argument.

In [ ]:
arr = np.array([[1., 2., 3.,], [4., 5., 6.]])
arr

### Array attributes

Arrays have a few other important attributes. Note attributes never have paretheses after them. Methods always do.

In [ ]:
arr.size          # The number of elements in the array

In [ ]:
arr.shape         # The shape of the array (i.e., the size of each dimension)

In [ ]:
arr.ndim          # The number of dimensions of the array

### Setting array shape

You can set the `array.shape` attribute to change the shape of the array. This attribute does not change the elements of the array, or how it is stored in memory, just how it is seen.

In [ ]:
arr.shape = (3, 2)
arr

In [ ]:
arr.shape = (6,)
arr

Singleton dimensions add to the dimensionality of an array. The last example was a 1D array, the next are 2D arrays.

In [ ]:
arr.shape = (1, 6)
arr   # Note that there are *two* square brackets in the output sequence.

In [ ]:
arr.shape = (6, 1)
arr   # this is also a 2D array, like a column vector

## Array indexing

Arrays are indexed in a similar way to sequences, with `start:stop:stride` notation, except that this is used for each dimension in the array. Colons denote all the values in a particular dimension, slices indicate some particular subset of the data in that particular dimension. 

A common use case is to get a single row or column from a 2D array (a table of data).

In [ ]:
arr = np.arange(60).reshape(6, 10)
arr

In [ ]:
arr[:, 4]   # the 5th column

In [ ]:
arr[2, :]   # the 3rd row

In [ ]:
arr[2]     # Trailing colons do not need to be explicitly typed. This is equivalent to the last example.

In [ ]:
arr[4, 7]   # an individual element in the table

---
### *Exercise*

> Slices can be combined in any way. Define a new array or use array `arr` and grab out every other row and the 4th column and above.

---

### Conventions concerning arrays containing spatio-temporal information

Generally, you will want to think of arrays as representing dimensions in space and time. The conventional way to think of this is that the dimensions are $(t, z, y, x)$; missing dimensions are omitted. This will help make plotting and analysis easier. Some examples might be:

    temp[:, :, :, :]     # A 4D array (time, height, latitude, longitude)
    press[:, :]          # A 2D array (time, height)
    humid[:, :]          # A 2D array (latitude, longitude)

## Array methods

Arrays have a number of methods. Let's take a look at the `mean` method as an example. 

In [ ]:
arr = np.array([[1., 2., 3.,], [4., 5., 6.]])  # reset the array to our 2x3 array.

arr.mean()        # The mean of all of the elements in the array

`Mean` takes the optional argument `axis` that can be used to take the mean along a single axis of the array. Just like with indexing, the axes are reference in a zero-based system; `axis=0` means the first dimension. 

In [ ]:
arr.mean(axis=0)  # The mean 

In this case, there are two rows in the first dimension, and `arr.mean(axis=0)` takes the average in the 'row' direction, resulting in a 1D array that is the average of each column.

---
### *Exercise*

> Find the mean of the array in the 'column' direction, along `axis=1`.

> Use the `sum` method of the array class to get the sum of the numbers in each column. The result should be a 1D array with three elements.

---

You can also use the `reshape` method to change the shape of an array.

In [ ]:
arr.reshape(3, 2)

You can find the mininum and maximum of an array with the `min` and `max` methods. Sometimes it is useful to find the indices of these minima and maxima. For this use `argmin` and `argmax`, like

In [ ]:
x = np.random.rand(10)
imax = x.argmax()
print(imax, x[imax], x.max())

## Array views

The data for an array may be stored in memory using `C` or `FORTRAN` ordered memory. Typically, there is no need to think about this, some details can be found [here](http://docs.scipy.org/doc/numpy-1.10.0/reference/internals.html).

However, it is important to remember that subsets of an array can produce a different 'view' of the array that addresses the same memory as the original array. This can lead to some unexpected behaviors. One way to think of this is that assignment in Python is more like a C-pointer (i.e., a reference to a memory location) than an actual value.

In [ ]:
a = np.arange(10.0)
b = a[::2]
print(a)
print(b)

In [ ]:
a[4] = -999   # this will modify b as well...
print(a)
print(b)

In [ ]:
b[-1] = -888  # this will modify a as well...
print(a)
print(b)

Normally, this will not be a problem, but if you need to make sure that a subset of an array has it's own memory, make sure you make a `copy` of the array, like

In [ ]:
a = np.arange(10.0)
b = a.copy()[::2]     # or np.copy(a)
a[4] = -999   # this will modify b as well...
print(a)
print(b)

## Vectorization

The best way to do mathematical operations using numpy arrays is to do `vector` operations. That is, mathematical operations are defined to be element by element, and this is done much faster than looping. As a rule of thumb, you should be very concerned if your code has more than one significant `for` loop in the numerical analysis section.

In [ ]:
a = np.arange(1024.0).reshape(4, 8, 16, 2)   # a 4D array using sequential numbers
b = np.random.rand(4, 8, 16, 2)              # a 4D array using random numbers

sol = a * b       # element-by-element multiplication. This operation is about as fast as it can be on your computer.

### Ufuncs

`Ufunc`s or Universal functions are ways to apply a function to every element in the array. Let's check [Euler's formula](https://en.wikipedia.org/wiki/Euler%27s_formula):

$e^{ix} = \cos(x) + i\sin(x) $

by using universal function `real` to see if the real part of the exponential really is equal to the real part of the right-hand side of the equation, $\cos(x)$.

In [ ]:
a = np.random.rand(3, 4, 5)  # create a random 3 x 4 x 5 array

res1 = np.exp(1.0J*a).real    # The `real` attribute returns the real part of a complex number
res2 = np.cos(a)

np.allclose(res1, res2)       # Checks if all of the elements are close, within some small tolerance.

## Array broadcasting

Arrays may be operated on using vector operations even if they are different sizes, however, they need to follow the rules of `array broadcasting`.  One way to think of this is that a larger dimension will be 'broadcast' across a singleton dimension. Generally, all of the dimensions need to be either the same size, or one of the dimension sizes for a particular dimension should be of size 1. Arrays always have as many 'singleton' dimensions to the left as needed.  For example, these arrays will all 'broadcast'

      a: 5 x 7 x 1 x 8
      b:             8
      c:     7 x 3 x 8
      d: 5 x 1 x 3 x 1
    
    sol: 5 x 7 x 3 x 8   

Let's create these arrays with random numbers

In [ ]:
a = np.random.rand(5, 7, 1, 8)
b = np.random.rand(8)
c = np.random.rand(7, 3, 8)
d = np.random.rand(5, 1, 3, 1)
print(a.shape, b.shape, c.shape, d.shape)

sol = a * b * c * d
print(sol.shape)

---
### *Exercise*

> Experiment with multiplying just two of the arrays together. Try to predict the resulting shape.

---

Array broadcasting sometimes requires creating new singleton dimensions in arrays. This can be done by putting `np.newaxis` in the appropriate space when indexing the array. For example

In [ ]:
x = np.arange(6)   # a 1D array. shape (6,)

x1 = x[:, np.newaxis]   # 2D array, shape (6, 1)
x2 = x[np.newaxis, :]   # 2D array, shape (1, 6)

np.abs(x1 - x2)  # A 'distance' matrix, a 2D array, shape (6, 6)

---
### *Exercise*

> Let's alter some of the previous arrays to instead have: `b = np.random.rand(8)` and `c = np.random.rand(8, 3, 7)`. What happens when you try to multiply these together? How can you fix it so it works?

---

## Combining and splitting arrays

Generally, arrays can be combined with the `np.concatenate` function. The arguments are a sequence of arrays to join, and the axis along which to join them (default=0).

There are a number of convenience functions that act like concatenate for specific axes:

 - `np.vstack` – vertical stack (stack along axis=0)
 - `np.hstack` – horizontal stack (stack along axis=1)
 - `np.dstack` – depth stack (stack along axis=2)



In [ ]:
x = np.random.rand(4, 5, 6)
y = np.random.rand(4, 5, 6)

print(np.concatenate((x, y)).shape)
print()
print(np.concatenate((x, y), axis=0).shape)
print(np.concatenate((x, y), axis=1).shape)
print(np.concatenate((x, y), axis=2).shape)
print()
print(np.vstack((x, y)).shape)
print(np.hstack((x, y)).shape)
print(np.dstack((x, y)).shape)

Likewise, arrays can be split with `np.split` or `np.array_split`. There are also convenience functions to split horizontally, vertically, and with depth.

In [ ]:
x = np.random.rand(12, 2, 5)
np.split(x, 4, axis=0)[2].shape

---
### *Exercise*

> Create a random array of shape (40, 50, 60). Split it along axis=1 five ways. Concatenate it along axis=2.

> What is the resulting shape?  _[Advanced: can you calculate this on one line?]_

---

---
### *Exercise*

> Concatenate newly-defined arrays `b` and `c`: b = np.random.rand(8) and c = np.random.rand(8, 3). Which of the above functions (`hstack`, `vstack`, `dstack`) would make sense with the arrays dimensions? Do you need to make any changes to get this to work?

---

## Flattening arrays with `a.flat` and `a.flatten()`

There are two basic ways to turn any array into a 1D array. They are slightly different.

`a.flatten()` returns a copy of an array, in one dimension.

In [ ]:
a = np.arange(12).reshape(3, 4)
print(a)
b = a.flatten()
print(b)

the `flat` attribute on the other hand gives a view of the array in 1D. It looks like an iterator object (like `range` and `zip`). This allows

In [ ]:
a.flat[6] = -999
print(a)

In contrast, this does not work as expected.  _WHY?_

In [ ]:
a.flatten()[5] = -888
print(a)

Other operations can be done to the array first. For example, we can take a transpose of the array before we flatten it.

In [ ]:
a.T.flat[6] = -999
print(a)

Here, the `T` attribute (equivalent to the `a.transpose()` method) gives a view of the array transposed (similar to MATLAB's tick notation).

In [ ]:
print(a.T)

## Creating standard arrays

There are a few standard arrays, for example, arrays filled with zeros or ones (or empty). Here are some examples of creating arrays

In [ ]:
o = np.ones((3, 4, 5))    # The argument is a shape, so is a tuple with the length of each dimension as an argument
b = np.ones((2, 3), dtype=np.bool)
z = np.zeros((2, 3), dtype=np.float32)

b

You can also create these arrays with the same shape and datatype of the input array using `np.ones_like` and `np.zeros_like`.

In [ ]:
zb = np.zeros_like(b)
zb

You can also create a diagonal array with a given vector along the diagonal. These can be offset with an optional argument `k` (default=0). This example creates a tri-diagonal array similar to that used for finite difference calculations

In [ ]:
np.diag(-2*np.ones(6)) + np.diag(np.ones(5), k=-1) + np.diag(np.ones(5), k=1)

There are also a number of ways to generate sequences of numbers.
 - `np.arange([start,] stop [[, stride]])` Create a sequence of numbers, similar to `range`
 - `np.linspace(min, max, length)` Create a uniform series of specified `length` between `min` and `max`, inclusive.
 - `np.logspace(minpow, maxpow, length)` Create a uniform series in logspace of specified `length` between `10**minpow` and `10**maxpow`, inclusive.
 

In [ ]:
np.arange(10)

In [ ]:
np.arange(2, 10, 2)

In [ ]:
np.linspace(2, 4, 17)

In [ ]:
np.logspace(-2, 2, 9)

## Finding values

There are a number of ways to find values in an array. The simplest is always to create a boolean array, like

In [ ]:
x = np.random.rand(5, 5)
print(x)
ind = x > 0.5
print(ind)

The boolean array can be used as an index to other arrays. Note this will return a 1D array, no matter what dimension the origial arrays are, because there is no way to know what structure the true values have.

In [ ]:
x = np.random.rand(5, 5)
y = np.sin(x)

y[x > 0.5]
# or, equivalently, as two lines
# idx = x > 0.5
# y[idx]

To get the indices of the places where the conditional is true (i.e., the locations of the `True` values in the boolean array), use the `np.where` command. 

In [ ]:
x = np.random.rand(5, 5)
idx = np.where(x > 0.5)
idx

Note that `np.where` ~always returns a tuple of indices for each dimension. This is a little strange for 1D arrays, but is done for consistency across all input values. Often, you will want to explicitly pull out the (single) array of indices from the tuple, like

In [ ]:
x = np.random.rand(10)
idx = np.where(x>0.5)[0]
print(idx)

_What happens with the [0] is missing behind the call to `where`?_

---
### *Exercise*

> You can also use these calculated indices, or boolean matricies on the left hand side for assignment.

> Create a 10x10 random array, with values between 0 and 1. Replace all of the numbers smaller than 0.5 with zero.

---

## Importing data

One of the basic commands in numpy for loading in data is the `loadtxt` command. There are other ways to do this, such as the [`genfromtxt`](http://docs.scipy.org/doc/numpy-dev/user/basics.io.genfromtxt.html) command, but `loadtxt` is sufficient for most purposes, and is easy to use.

In [ ]:
data = np.loadtxt('../data/CTD.txt', comments='*')
data[:,2]    # a column of data representing temperature

## Linear algebra

One of the key elements of the `numpy` package is the `numpy.linalg` subpackage that contains a number of linear algebra functions that work efficiently on arrays.

In [ ]:
a = np.random.randn(100, 100)
e, v = np.linalg.eig(a)

b = np.random.randn(500, 200)
u, s, v = np.linalg.svd(b)

Matrix multiplication is done using the `np.dot` function. In this case, matrices do _not_ need to be the same shape, but must follow the rules of matrix multiplication. E.g., the operation dot(<4x5 array>, <5x12 array>) results in a 4x12 array; i.e., the inner dimensions must match (technically last and second-to-last, for arrays with more than two dimensions).   

In [ ]:
x = np.random.rand(4, 5)
y = np.random.rand(5, 12)

res = np.dot(x, y)
print(res.shape)

# np.dot(y, x)  # This gives an error -- order is important.

## Polynomial fitting

The basic function for fitting a polynomial (e.g., a straight line) is `np.polyfit(x, y, deg)`. There are a number of other functions that let you add (`np.polyadd`), multiply (`np.polymul`), find zeros (`np.roots`), and do other operations to polynomials.

In [ ]:
x = np.random.rand(100)
y = 5 + 3*x + 0.1*np.random.randn(100)   # A line with some noise

p = np.polyfit(x, y, 1)
print(p)  # The coefficients of the polynomial, with highest order first. (i.e,. [slope, intercept])

You can also use the `np.polynomial.Polynomial` class to work with polynomials. Note, these define polynomials the opposite way, with the _lowest_ order first. The Polynomial class gives an excellent example of operator overloading, and the flexibility of classes.

In [ ]:
p1 = np.polynomial.Polynomial([5, 3])         # y = 5 + 3 x
p2 = np.polynomial.Polynomial([3, 6, 8, 2])   # y = 3 + 6 x + 8 x**2 + 2 x**3

print('Evaluation')
print('p1(0.0) = ', p1(0))
print('p1(5.0) = ', p2(5))
print()
print('Roots')
print('Roots of p2 = ', p2.roots())
print()
print('Operations')
print('p1 + p2 = ', p1 + p2)
print('p1 * p2 = ', p1 * p2)
print()
print('Calculus')
print('Derivative of p1', p1.deriv(1))
print('Inegral of p2', p2.integ(4, k=[4, 3, 2, 1]))

## Basic performance evaluation

We can do some very basic perfomance testing using the `%time` special function in jupyter notebooks. Lets use this to examine the time it takes to do a singular value decomposition for different sized matricies.

In [ ]:
b = np.random.randn(500, 200)
%time u, s, v = np.linalg.svd(b)

If the time might change, say based on the values chosen, the `%timeit` function can be used to perform the test a number of times to get an average calculation time.

In [ ]:
%timeit b = np.random.randn(50, 20); u, s, v = np.linalg.svd(b)

For statements that are longer than a single line, the `time.time` function can be used.

In [ ]:
import time

t_start = time.time()
time.sleep(0.25)   # Do nothing for 0.25 seconds
t_stop = time.time()

print('{:6.4f} seconds have passed.'.format(t_stop-t_start))

## Masked arrays

Masked arrays are ways to create arrays with missing values. MATLAB&trade; uses NaNs (NaN stands for 'Not a Number'), and the NaNs are the values of the arrays at those points. This approach also works in Python. Masked arrays are preferred since they retain the masked array vaules, and also some plotting routines require masked arrays when plotting arrays with missing values. Masked arrays are usually created through some condition, like

In [ ]:
arr = np.random.randn(7, 8)
cond = np.random.rand(7, 8) > 0.5   # `cond` is True for the random values greater than 0.5

marr = np.ma.masked_where(cond, arr)

print(marr)

np.allclose(marr.data, arr)

The mask can also be supplied explicity when creating the masked array,

In [ ]:
marr = np.ma.masked_array([1, 2, 3, 4, 5], mask=[True, True, False, False, True])
marr

## Overview of scipy packages

The `scipy` package contains a number of specialized numerical computational tools. These tools are usually very specific, and in the case of the linear algebra tools, may be optimized for your particular hardware.

     cluster                      --- Vector Quantization / Kmeans
     fftpack                      --- Discrete Fourier Transform algorithms
     integrate                    --- Integration routines
     interpolate                  --- Interpolation Tools
     io                           --- Data input and output
     lib                          --- Python wrappers to external libraries
     lib.lapack                   --- Wrappers to LAPACK library
     linalg                       --- Linear algebra routines
     misc                         --- Various utilities that don't have
                                      another home.
     ndimage                      --- n-dimensional image package
     odr                          --- Orthogonal Distance Regression
     optimize                     --- Optimization Tools
     signal                       --- Signal Processing Tools
     sparse                       --- Sparse Matrices
     sparse.linalg                --- Sparse Linear Algebra
     sparse.linalg.dsolve         --- Linear Solvers
     sparse.linalg.dsolve.umfpack --- :Interface to the UMFPACK library:
                                      Conjugate Gradient Method (LOBPCG)
     sparse.linalg.eigen.lobpcg   --- Locally Optimal Block Preconditioned
                                      Conjugate Gradient Method (LOBPCG) [*]
     special                      --- Airy Functions [*]
     lib.blas                     --- Wrappers to BLAS library [*]
     sparse.linalg.eigen          --- Sparse Eigenvalue Solvers [*]
     stats                        --- Statistical Functions [*]
     lib                          --- Python wrappers to external libraries
                                      [*]
     lib.lapack                   --- Wrappers to LAPACK library [*]
     integrate                    --- Integration routines [*]
     ndimage                      --- n-dimensional image package [*]
     linalg                       --- Linear algebra routines [*]
     spatial                      --- Spatial data structures and algorithms
     special                      --- Airy Functions
     stats                        --- Statistical Functions


In [ ]:
import scipy   # This actually does nothing. Submodules must be explicitly imported

## `scipy.stats`

In [ ]:
import scipy.stats

In [ ]:
rv = scipy.stats.laplace(loc=0, scale=1)
rv.pdf(np.linspace(-3, 3, 101))    # create a pdf of the laplace distribution with parameters loc=0, scale=1

In [ ]:
r = scipy.stats.laplace.rvs(size=100) # Generate 100 random numbers based on the laplace distribution
r

## `scipy.integrate`

In [ ]:
import scipy.integrate

sigma=10.0
beta=8.0/3.0
rho=28.0

def lorenz(state, to):
    x, y, z = state
    xdot = sigma*(y-x)
    ydot = x*(rho-z)-y
    zdot = x*y - beta*z
    return (xdot, ydot, zdot)
    
t = np.linspace(0, 30, 3000)
sol = scipy.integrate.odeint(lorenz, (1., 2., 3.), t)
x, y, z = sol.T

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
% matplotlib inline

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.plot(x, y, z)

---
### *Exercise*

> Read in again the oceanographic data file '../data/CTD.txt' into an array. You can look at the data file itself to see what variables are stored in each column.

> Using this data, write a function to calculate the linear equation of state. This is an approximation of the density of water, as it depends on salinity, temperature, and some empirical constants. We will use the following form for the linear equation of state:

> $\rho = 1027[1+7.6\times 10^{-4}(S-35) -1.7\times 10^{-4}(T-25)]$

> where $\rho$ is the density, $S$ is the salinity, and $T$ is the temperature.

> This is more free form than the homework, so you should set up all of the associated code to call the function, and write out the function yourself. Don't forget docstrings! For a check, the first value of your density array in order should equal 1021.7519981630001 and the last should equal 1028.0471353619998.

---

---
### *Exercise*

> Output from a numerical model of the northwestern Gulf of Mexico are saved in a file `../data/model.npz`. Read in this file. Among other things, it contains `h`, the depths within the numerical domain, and `ssh`, the sea surface heights at two time steps. The sea surface height gives the deviation above and below sea level from a reference water level (which changes in time as the water moves), and the depths of the seabed are also given with respect to that reference water level. 

> Find the full water column depth, between the seabed and the sea surface, for the two given times. 

> You can use as a comparison that at the first time step the [0,0] value of this array should be 3007.6088347392124, and at the second time step the [0,-1] value should be 605.25282427018749. Note that there is a differences between the two time steps though it is generally quite small since it is the difference between time steps in the numerical circulation model.

---

---
### *Exercise*

> Earlier, we discussed using array operations instead of looping because it is faster. Let's compare.

> Calculate the time it takes to do what we did before:

    a = np.arange(1024.0).reshape(4, 8, 16, 2)   # a 4D array using sequential numbers
    b = np.random.rand(4, 8, 16, 2)              # a 4D array using random numbers
    sol = a * b  # element-by-element multiplication. This operation is about as fast as it can be on your computer.

> and the time required for doing the same operation with a series of 4 `for` loops, one for each dimension of the arrays. Compare the times by calculating a ratio.

---